In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import *
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append('../Data/')

In [ ]:
#This part is for you to use inside your DataSet subclass
cereals = pd.read_csv('/content/drive/MyDrive/DATA/cereal.csv')
cereals_numeric = cereals.drop(['name', 'mfr', 'type'], axis=1, inplace=True)
cereals.columns

Index(['calories', 'protein', 'fat', 'sodium', 'fiber', 'carbo', 'sugars',
       'potass', 'vitamins', 'shelf', 'weight', 'cups', 'rating'],
      dtype='object')

You can get help from the code above and use it in your DataSet class

In [ ]:
def train_simple_network_ce(model, loss_func, training_loader, epochs=20, device="cpu"):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    model.to(device)

    for epoch in tqdm(range(epochs), desc="Epoch"):
        model.train()
        running_loss = 0.0

        for inputs, labels in tqdm(training_loader, desc="Batch", leave=False):
            inputs = inputs.to(device)

            # BCELoss needs float targets (0/1) and matching shape
            labels = labels.to(device)

            optimizer.zero_grad()

            logits = model(inputs)                  # raw output, shape [B,1] usually

            loss = loss_func(logits, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / max(1, len(training_loader))
        tqdm.write(f"Epoch {epoch+1}/{epochs} - loss: {avg_loss:.4f}")

In [ ]:
class DataSet():
  def __init__(self, filename):
    self.filename=filename
    self.cereals = pd.read_csv(filename)
    self.cereals_numeric = cereals.drop(['name', 'mfr', 'type'], axis=1, inplace=True)
    cereals.columns

  def __getitem__(self, index):
    return torch.tensor(self.cereals_numeric[index,:], dtype=torch.float32), torch.tensor(self.cereals_numeric[index], dtype=torch.long)

# Ok, so it can't find the data dir so I can't test this, so be kind. It should work though
X, y = DataSet(('/content/drive/MyDrive/DATA/cereal.csv'))
DataLoader((X, y), shuffle=True)

model = nn.Sequential(
    nn.Linear(4,  10),
    nn.ReLU(),
    nn.Linear(10,  10),
    nn.ReLU(),
    nn.Linear(10, 3),
)
loss = nn.CrossEntropyLoss()
train_simple_network_ce(model, loss, training_loader, epochs=20, device="cpu")


You can use the code snippet below to plot your loss

In [ ]:
sns.lineplot(x=range(len(loss)), y=loss)